### Necessary imports

In [ ]:
!pip install -q -U torch datasets transformers==4.36.1 tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline


from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

### Load quantized Mistal 7B

In [ ]:
from huggingface_hub import login
login("*************************")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.2'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### Count number of trainable parameters

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


### Build Mistral text generation pipeline

In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [ ]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### Load and chunk documents. Load chunked documents into FAISS index

In [ ]:
!playwright install
!playwright install-deps

152.8 MiB [] 0% 0.0s152.8 MiB [] 0% 34.4s152.8 MiB [] 0% 20.1s152.8 MiB [] 0% 11.7s152.8 MiB [] 0% 10.5s152.8 MiB [] 1% 7.4s152.8 MiB [] 1% 6.1s152.8 MiB [] 2% 5.3s152.8 MiB [] 2% 5.1s152.8 MiB [] 2% 5.2s152.8 MiB [] 3% 5.0s152.8 MiB [] 3% 4.6s152.8 MiB [] 4% 4.6s152.8 MiB [] 4% 4.4s152.8 MiB [] 5% 4.2s152.8 MiB [] 5% 4.1s152.8 MiB [] 6% 4.0s152.8 MiB [] 7% 3.9s152.8 MiB [] 8% 3.7s152.8 MiB [] 8% 3.5s152.8 MiB [] 9% 3.6s152.8 MiB [] 9% 3.5s152.8 MiB [] 10% 3.5s152.8 MiB [] 10% 3.3s152.8 MiB [] 11% 3.2s152.8 MiB [] 12% 3.1s152.8 MiB [] 13% 3.0s152.8 MiB [] 13% 2.9s152.8 MiB [] 14% 2.9s152.8 MiB [] 15% 2.9s152.8 MiB [] 16% 2.8s152.8 MiB [] 17% 2.7s152.8 MiB [] 18% 2.6s152.8 MiB [] 19% 2.5s152.8 MiB [] 21% 2.4s152.8 MiB [] 21% 2.3s152.8 MiB [] 23% 2.2s152.8 MiB [] 24% 2.1s152.8 MiB [] 25% 2.0s152.8 MiB [] 26% 2.0s152.8 MiB [] 27% 1.9s152.8 MiB [] 28% 1.9s152.8 MiB [] 29% 1.9s152.8 MiB [] 30% 1.8s152.8 MiB [] 31% 1.7s152.8 MiB [] 32% 1.7s152.8 MiB [] 33% 1.7s152.8 MiB [] 34% 1.6s152.8 MiB 

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index

articles = ["https://es.euronews.com/tag/inteligencia-artificial"]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [ ]:
# Converts HTML to plain text
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=1000,
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

### Create PromptTemplate and LLMChain

In [ ]:
prompt_template = """
### [INST] Instrucción: Eres un experto en inteligecia artificial responda en español la pregunta según sus conocimientos de la siguiente página web. Aquí hay contexto para ayudar:

{context}

### PREGUNTA:
{question} (responde en castellano) [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
llm_chain.invoke({"context": "", "question": " Dime los titulares y una pequeña descripción de las noticias hay relacionadas con la Inteligencia artificial (las que no traten de IA no las quiero)"})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': '',
 'question': ' Dime los titulares y una pequeña descripción de las noticias hay relacionadas con la Inteligencia artificial (las que no traten de IA no las quiero)',
 'text': ' I. "IA asesora médica: cómo la inteligencia artificial está revolucionando el diagnóstico y tratamiento de enfermedades"\n    Descripción: Se habla sobre cómo la inteligencia artificial se está utilizando en el campo de la medicina para mejorar el diagnóstico y tratamiento de diversas enfermedades, reduciendo errores humanos y aumentando la eficiencia.\n\n  II. "El futuro de la inteligencia artificial: tendencias y desafíos"\n    Descripción: Se analizan las últimas tendencias en el mundo de la inteligencia artificial y se discuten los principales desafíos que siguen por superar para avanzar en este campo.\n\n  III. "Inteligencia artificial en el comercio electrónico: cómo las empresas aprovechan esta tecnología"\n    Descripción: Se explica cómo la inteligencia artificial se utiliza en el comerc

### Build RAG Chain

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("¿Qué noticias hay? dime los titulares y una pequeña descripción")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
result['context']

[Document(page_content='###  Una española será la primera mujer en casarse con un holograma\n\nLa artista multidisciplinar barcelonesa Alicia Framis será la primera mujer en\ncontraer "matrimonio" con un holograma generado por Inteligencia Artificial en\nuna ceremonia que tendrá lugar este verano en el museo Depot Boijmans Van\nBeuningen de Róterdam\n\n17/12/2023\n\nReproduciendo  Siguiente\n\nReal Economy\n\n###  ¿Qué significará la IA para el futuro de los trabajadores europeos?\n\nEn este episodio de Economía Real nos preguntamos qué significa la\ninteligencia artificial para el mundo laboral. ¿Podrá facilitarte esas tareas\ntediosas y complicadas o, de hecho, vendrá a por tu trabajo? Y si aún no estás\nen la línea de fuego, ¿podrías trabajar pronto junto a un robot?\n\n14/12/2023\n\nReproduciendo  Siguiente\n\nMercados\n\n###  ¿Confiaría sus finanzas a un robot?', metadata={'source': 'https://es.euronews.com/tag/inteligencia-artificial'}),
 Document(page_content='###  Bruselas revi

In [ ]:
resultado = result['text']

In [ ]:
print(resultado)

1. **"Una española será la primera mujer en casarse con un holograma":** Alicia Framis, una artista multidisciplinar de Barcelona, será la primera mujer en contraer matrimonio con un holograma creado por Inteligencia Artificial (IA). Esto sucederá en el verano de 2023 en el museo Depot Boijmans Van Beuningen de Róterdam.

 2. **"¿Qué significará la IA para el futuro de los trabajadores europeos?":** En este episodio de Economía Real, se discute qué significa la IA para el mundo laboral. Se pregunta si podrá hacer tus tareas más sencillas o incluso te robará el trabajo.

 3. **"Bruselas revisará la inversión de Microsoft en OpenAI":** La Comisión Europea anunciará si la inversión de Microsoft en OpenAI entrará en el ámbito de aplicación de las leyes sobre fusiones del bloque.

 4. **"La IA roba el protagonismo en la feria tecnológica de Las Vegas":** El futuro de la IA abarca todos los aspectos de la vida, desde monitorear la salud en tiempo real hasta el uso de psicólogos falsos en el 